# Imports 

In [1]:
import cv2
import os
import numpy as np
import json
from kafka import KafkaProducer, KafkaConsumer
from cassandra.cluster import Cluster
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import schedule
import time
import base64
import io
import requests
from keras.applications.vgg16 import VGG16  # Assuming you want to use Keras VGG16
from cassandra.util import uuid_from_time

# configuration

In [2]:
# Kafka configuration
bootstrap_servers = 'localhost:9092'
input_topic = 'photo-topic'
output_topic = 'output_topic_net'

# Cassandra configuration
cassandra_host = 'localhost'
cassandra_keyspace = 'Photos'
cassandra_table = 'photosFires'

# Connect to Kafka
consumer = KafkaConsumer(bootstrap_servers=bootstrap_servers)
consumer.subscribe([input_topic])

# Kafka producer
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)


# Load the saved model
save_model = tf.keras.models.load_model(r'C:\Users\AL-FAJR\Desktop\New folder (3)\Last_GP\GP_Model\my_keras_model.h5')
class_names = ['fire_images', 'non_fire_images']
#image_folder = r"C:\Users\AL-FAJR\Desktop\Last_GP\FIRE-SMOKE-DATASET\Test\Neutral"

# Cassandra

In [3]:
from cassandra.cluster import Cluster

# Connect to Cassandra cluster
cluster = Cluster(['127.0.0.1'])  # Provide contact points for your Cassandra cluster
session = cluster.connect()

# Define keyspace and table names
cassandra_keyspace = 'your_keyspace_name'
cassandra_table = 'your_table_name'

# Create keyspace if it doesn't exist
session.execute(f"CREATE KEYSPACE IF NOT EXISTS {cassandra_keyspace} WITH REPLICATION = {{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }}")

# Set the keyspace
session.set_keyspace(cassandra_keyspace)

# Create table if it doesn't exist
session.execute(f"CREATE TABLE IF NOT EXISTS {cassandra_table} (id UUID ,homeId INT,isRead Text,roomId INT, class_name TEXT , image_Path TEXT , DateTime TIMESTAMP, PRIMARY KEY (homeid,id))")


# Email

In [4]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_email(receiver_email):
    # Email configuration
    

    # Create message
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = "Fire Detect In Your Home"

    # Email body
    body = "Fire Detect In Your Home!"
    message.attach(MIMEText(body, "plain"))

    # Connect to SMTP server
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message.as_string())

    print("Email sent successfully!")

# Call the function with the receiver email as an argument
#receiver_email = "mohamed.essam.anwar2001@gmail.com"
#send_email(receiver_email)



# requests

In [5]:
import requests

def get_data(param_value):
    # Define the API endpoint URL
    api_url = 'http://localhost:62863/Api/Home/GetMl'

    # Make the API request with the parameter included in the URL
    response = requests.get(f"{api_url}/{param_value}")

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Extract phoneNumber and email values
        phone_number = data.get('phoneNumber')
        email = data.get('email')

        # Return the extracted values
        return phone_number, email
    else:
        # Print the error message
        print(f"Error: {response.status_code} - {response.text}")
        return None, None

# Example usage:
#param_value = 1
#phone_number, email = get_data(param_value)
#if phone_number is not None and email is not None:
   # print("Phone Number:", phone_number)
   # print("Email:", email)


# twilio

In [6]:
from twilio.rest import Client

def send_sms(recipient_phone_number):
    # Twilio account credentials
    message = 'Fire detected!'
    

    # Twilio phone number (bought or verified in Twilio)
    twilio_phone_number = "+14139615530"  # Your Twilio phone number

    # Initialize Twilio client
    client = Client(account_sid, auth_token)

    # Format recipient's phone number with Egypt country code
    recipient_phone_number = '+2' + recipient_phone_number

    # Send SMS
    message = client.messages.create(
        body=message,
        from_=twilio_phone_number,
        to=recipient_phone_number
    )

    print("SMS sent successfully!")

# Example usage:
#message = 'Hello from Twilio!'
#recipient_phone_number = '1234567890'  # Replace with recipient's phone number
#send_sms(message, recipient_phone_number)


{"HomeId":1,"RoomId":1,"DateTime":"2024-02-09T12:00:00","ImagePath": "C:\\Users\\TEMP\\Desktop\\image_0.jpg"}

# Kafka

In [ ]:
# Consume messages from Kafka
for message in consumer:
    try:
        # Check if the message is empty
        if not message.value:
            print("Ignoring empty message.")
            continue

        # Print the content of the message
        print("Received message:", message.value)

        # Parse message as JSON
        data_dict = json.loads(message.value.decode('utf-8'))

        # Extract fields from the JSON
        HomeId = data_dict['HomeId']
        RoomId = data_dict['RoomId']
        DateTime = data_dict['DateTime']
        ImagePath = data_dict['ImagePath']

        # Read image from folder
        image_path = os.path.join(data_dict['ImagePath'])
        image = Image.open(image_path)

        # Preprocess image for VGG16
        image = image.resize((224, 224))
        image = image.convert('RGB')
        image_array = preprocess_input(np.array(image))
        image_array = np.expand_dims(image_array, axis=0)

        # Classify the image
        prediction = save_model.predict(image_array)
        class_index = np.where(prediction >= 0.5, 1, 0)[0][0]
        class_name = class_names[class_index]
        probability = float(prediction)
        if (class_name=="fire_images"):
            IsRead = "0" 
            session.execute(f"INSERT INTO {cassandra_table} (id,homeId,isRead ,roomId, class_name, image_Path, DateTime) VALUES (uuid(),{HomeId},'{IsRead}',{RoomId}, '{class_name}', '{image_path}', '{DateTime}')")
            phone_number, email = get_data(HomeId)
            send_email(email)
            #send_sms(phone_number)
         # query = f"INSERT INTO {cassandra_table} (id, homeId, isRead, roomId, class_name, image_Path, DateTime) VALUES (uuid(), ?, ?, ?, ?, ?, ?)"
         # print("Query:", query)
         # print("Arguments:", (HomeId, IsRead, RoomId, class_name, image_path, DateTime))
         # session.execute(query, (HomeId, IsRead, RoomId, class_name, image_path, DateTime))
        
        # Publish data to another Kafka topic
        data_dict['probability'] = probability
        data_dict['class_name'] = class_name
        #data_dict['IsRead'] = IsRead

        # Convert the dictionary to JSON
        json_data = json.dumps(data_dict)

        # Send the JSON data to the output topic
        producer.send(output_topic, value=json_data.encode('utf-8'))

        print("Processed message:", data_dict)

    except Exception as e:
        print("Error processing message:", e)

# Close Kafka producer and consumer
producer.close()
consumer.close()


Received message: b'{"HomeId":1,"RoomId":1,"DateTime":"2024-02-09T12:00:00","ImagePath": "C:\\\\Users\\\\AL-FAJR\\\\Desktop\\\\New folder (3)\\\\Last_GP\\\\FIRE-SMOKE-DATASET\\\\Test.2\\\\image_71.jpg"}'
1/1 [==============================] - 1s 535ms/step
Processed message: {'HomeId': 1, 'RoomId': 1, 'DateTime': '2024-02-09T12:00:00', 'ImagePath': 'C:\\Users\\AL-FAJR\\Desktop\\New folder (3)\\Last_GP\\FIRE-SMOKE-DATASET\\Test.2\\image_71.jpg', 'probability': 1.0, 'class_name': 'non_fire_images'}
